<a href="https://colab.research.google.com/github/hkd7148-blip/ml-portfolio/blob/main/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets accelerate sentencepiece evaluate rouge_score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00


In [ ]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast
from datasets import load_dataset
import evaluate



Transformers и datasets импортируются ок ✅


In [ ]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration

model_name = "t5-small"
tokenizer = T5TokenizerFast.from_pretrained(model_name)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
"muennighoff/xsum"


'muennighoff/xsum'

In [ ]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")
print(dataset)


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

3.0.0/validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [ ]:
dataset["train"][0]


{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [ ]:
dataset = load_dataset("cnn_dailymail", "3.0.0")


In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess_function(batch):
    inputs = ["summarize: " + doc for doc in batch["article"]]
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True
    )

    labels = tokenizer(
        batch["highlights"],
        max_length=max_target_length,
        truncation=True
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset["train"][0]


{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [ ]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("t5-small")
model


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
from transformers import TrainingArguments


In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size = 2
epochs = 1
learning_rate = 3e-4
weight_decay = 0.01

training_args = Seq2SeqTrainingArguments(
    output_dir="t5-small-finetuned-cnn",
    do_train=True,
    do_eval=True,

    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    learning_rate=learning_rate,
    weight_decay=weight_decay,

    num_train_epochs=epochs,
    save_total_limit=2,

    predict_with_generate=True,
    generation_max_length=128,

    fp16=True,
    logging_steps=50,
)

training_args


Seq2SeqTrainingArguments(output_dir='t5-small-finetuned-cnn', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, eval_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, torch_empty_cache_steps=None, learning_rate=0.0003, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, lr_scheduler_kwargs={}, warmup_ratio=0.0, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='t5-small-finetuned-cnn/runs/Nov15_15-19-15_0826fed66009', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=50, logging_nan_inf_filter=True, save_strategy=<SaveStrategy.STEPS: 'st

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding="longest"
)

data_collator


DataCollatorForSeq2Seq(tokenizer=T5TokenizerFast(name_or_path='t5-small', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id

In [ ]:
import evaluate
rouge = evaluate.load("rouge")


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Декодируем предсказания
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Заменяем -100 на pad_token_id в labels
    labels = [[(l if l != -100 else tokenizer.pad_token_id) for l in label] for label in labels]

    # Декодируем правильные ответы
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Считаем ROUGE
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )

    # Усредняем длину предсказаний
    prediction_lens = [len(pred.split()) for pred in decoded_preds]
    result["gen_len"] = sum(prediction_lens) / len(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer


/tmp/ipython-input-1222785013.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
import os
if "WANDB_DISABLED" in os.environ:
    del os.environ["WANDB_DISABLED"]


In [ ]:
training_args = TrainingArguments(
    output_dir="t5-small-finetuned-cnn",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=epochs,
    weight_decay=weight_decay,
    logging_steps=50,
    save_strategy="epoch",
    eval_strategy="epoch",
    fp16=True,
    report_to="none",
)


In [ ]:
from transformers import Seq2SeqTrainingArguments


In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size = 2
epochs = 1
learning_rate = 3e-4
weight_decay = 0.01

training_args = Seq2SeqTrainingArguments(
    output_dir="t5-small-finetuned-cnn",
    do_train=True,
    do_eval=True,

    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    learning_rate=learning_rate,
    num_train_epochs=epochs,
    weight_decay=weight_decay,

    logging_steps=50,
    predict_with_generate=True,

    fp16=True,
    report_to="none"   # полностью отключаем внешние логгеры (W&B и пр.)
)

training_args


Seq2SeqTrainingArguments(output_dir='t5-small-finetuned-cnn', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, eval_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, torch_empty_cache_steps=None, learning_rate=0.0003, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, lr_scheduler_kwargs={}, warmup_ratio=0.0, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='t5-small-finetuned-cnn/runs/Nov15_15-27-49_0826fed66009', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=50, logging_nan_inf_filter=True, save_strategy=<SaveStrategy.STEPS: 'st

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer


/tmp/ipython-input-439360356.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
small_train = dataset["train"].select(range(1000))
small_valid = dataset["validation"].select(range(200))

dataset_small = {
    "train": small_train,
    "validation": small_valid
}


In [ ]:
trainer.train()


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
tokenized_dataset = {
    "train": dataset_small["train"].map(
        preprocess_function,
        batched=True,
        remove_columns=dataset_small["train"].column_names
    ),
    "validation": dataset_small["validation"].map(
        preprocess_function,
        batched=True,
        remove_columns=dataset_small["validation"].column_names
    )
}

len(tokenized_dataset["train"]), len(tokenized_dataset["validation"])


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

(1000, 200)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer


/tmp/ipython-input-3988066250.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()


Step,Training Loss
50,2.231700
100,2.198900
150,2.275500
200,2.089000
250,2.237200
300,2.155200
350,2.119200
400,2.209000
450,2.220700
500,2.141800


TrainOutput(global_step=500, training_loss=2.187807678222656, metrics={'train_runtime': 2527.8904, 'train_samples_per_second': 0.396, 'train_steps_per_second': 0.198, 'total_flos': 134474768056320.0, 'train_loss': 2.187807678222656, 'epoch': 1.0})

In [ ]:
metrics = trainer.evaluate()
metrics


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 2.151066303253174,
 'eval_rouge1': 0.2527,
 'eval_rouge2': 0.1017,
 'eval_rougeL': 0.2069,
 'eval_rougeLsum': 0.2061,
 'eval_gen_len': 13.205,
 'eval_runtime': 326.3353,
 'eval_samples_per_second': 0.613,
 'eval_steps_per_second': 0.306,
 'epoch': 1.0}

In [ ]:
sample = dataset["validation"][0]["article"]

inputs = tokenizer(
    "summarize: " + sample,
    return_tensors="pt",
    truncation=True,
    max_length=512
).to(model.device)

summary_ids = model.generate(
    **inputs,
    max_length=50,
    min_length=5,
    num_beams=4
)

print(tokenizer.decode(summary_ids[0], skip_special_tokens=True))


"I thought I was going to help this one person who I don't know, but the fact that so many people can have a life extension, that's pretty big," says Zully Broussard. Five surgeon
